# Extract and crop the interpreter

In [ ]:
import numpy as np
import os
import sys

notebook_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_dir, '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

%load_ext autoreload
%autoreload 2


def crop_interpreter(img: np.ndarray):
    h, w = img.shape[:2]

    if h == 1080:
        X = 1522
        Y = 720
        W = 383
        H = 411
    elif h == 720:
        X = 1015
        Y = 480
        W = 255
        H = 274
    else:
        raise ValueError('Height not supported')

    return img[Y : Y + H, X : X + W]


ROOT_DIR = 'scraped'

## Get frame indexes between which the interpreter is present

In [ ]:
import numpy as np
from cv2 import VideoCapture
import face_recognition

from utils.video import get_frame, get_vid_metadata

known_faces = []
known_people = []
for file_name in os.listdir('known_people'):
    img = face_recognition.load_image_file(os.path.join('known_people', file_name))
    enc = face_recognition.face_encodings(img)[0]
    known_faces.append(enc)
    known_people.append(file_name.replace('.png', ''))


def is_known(idx: int, cap: VideoCapture):
    """Wether we recognize one person in the interpreter box or not"""
    frame = get_frame(idx, cap)
    frame = crop_interpreter(frame)
    encodings = face_recognition.face_encodings(frame)
    for enc in encodings:
        matches = face_recognition.compare_faces(known_faces, enc)
        if any(matches):
            return known_people[np.argmax(matches)]
    return None


# false, ..., false, true, ..., true, false, ..., false
def bsearch(l: int, r: int, predicate):  # noqa: E741
    """Binary search on a sequence like [false, ..., false, true, ..., true]. Returns index of first true.
    Helps optimally search for the start and end indexes where the interpreter is present"""
    res = r
    while l <= r:
        m = (l + r) // 2
        if predicate(m):
            res = m
            r = m - 1
        else:
            l = m + 1  # noqa: E741
    return res

In [ ]:
import json
import os


def do_segment(path: str):
    cap, fps, frame_count = get_vid_metadata(path)
    frame_count -= 1
    stride = (
        fps * 60 * 15
    )  # 30 min of SL at least (required by law) but let's search in chunks of 15 mins so we don't jump over if a tiny bit less than 30 mins
    stride = min(stride, frame_count // 4)
    true_idx = -1
    interpreter = None
    for i in range(0, frame_count, int(stride)):
        interpreter = is_known(i, cap)
        if interpreter:
            true_idx = i
            break
    if true_idx == -1:
        return None

    start = bsearch(0, true_idx, lambda x: is_known(x, cap))
    first_false = bsearch(true_idx, frame_count - 1, lambda x: not is_known(x, cap))

    if first_false < frame_count and not is_known(first_false, cap):
        end = first_false - 1
    else:
        end = frame_count - 1

    return start, end, interpreter


for root, _, files in os.walk(ROOT_DIR):
    for file in files:
        vid = os.path.join(root, file)
        if file.endswith('.mp4'):
            seg = vid.replace('.mp4', '.json')
        elif file.endswith('.mkv'):
            vid = os.path.join(root, file)
            seg = vid.replace('.mkv', '.json')
        else:
            continue
        if os.path.exists(seg):
            print(f'Skipping {seg}')
            continue
        segment = do_segment(vid)
        if not segment:
            print(f'no segment for {vid}')
            continue
        with open(seg, 'w') as f:
            json.dump(
                {'start': segment[0], 'end': segment[1], 'interpreter': segment[2]}, f
            )
        print(seg)

### Sometimes the interpreter disappears for some time and reappears quite later. Do find them out

In [ ]:
import json
import os


# Reuse the do segment logic but apply it BEFORE the identified segment and AFTER the identified segment
# (Originally this function was used to FIND the segment. Now see if there are some other, weird segments
# we missed.)
def do_segment_weird(path: str, startx: int, endx: int):
    cap, fps, frame_count = get_vid_metadata(path)
    frame_count -= 1
    stride = fps * 60 * 5  # 5 min
    stride = min(stride, frame_count // 12)
    true_idx = -1
    interpreter = None
    if startx == -1:
        startx = 0
    if endx == -1:
        endx = frame_count
    for i in range(startx, endx, int(stride)):
        interpreter = is_known(i, cap)
        if interpreter:
            true_idx = i
            break
    if true_idx == -1:
        return None

    start = bsearch(0, true_idx, lambda x: is_known(x, cap))
    first_false = bsearch(true_idx, frame_count - 1, lambda x: not is_known(x, cap))

    if first_false < frame_count and not is_known(first_false, cap):
        end = first_false - 1
    else:
        end = frame_count - 1

    return start, end, interpreter


for root, _, files in os.walk(ROOT_DIR):
    for file in files:
        vid = os.path.join(root, file)
        if file.endswith('.mp4'):
            seg = vid.replace('.mp4', '.json')
        elif file.endswith('.mkv'):
            vid = os.path.join(root, file)
            seg = vid.replace('.mkv', '.json')
        else:
            continue
        with open(seg) as f:
            seg = json.load(f)
        if isinstance(seg, list):
            continue
        print(f'trying {file}')
        segment = do_segment_weird(vid, -1, seg['start'] - 1)
        if segment:
            print(file + ' BEFORE')
        segment = do_segment_weird(vid, seg['end'] + 1, -1)
        if segment:
            print(file + ' AFTER')

## Some stats

In [7]:
sum = 0

for root, _, files in os.walk(ROOT_DIR):
    for file in files:
        if not file.endswith('.json'):
            continue
        with open(os.path.join(root, file)) as f:
            j = json.load(f)
            if isinstance(j, list):
                for x in j:
                    sum += x['end'] - x['start']
            else:
                sum += j['end'] - j['start']

print(sum / 25 / 60 / 60)

284.80795555555557


## Crop segments with ffmpeg

In [ ]:
import os
import json
import shlex
import subprocess


ENCODER_TYPE = 'cpu'  # Options: "nvidia", "cpu"
QUALITY_SETTING = 20  # 0 is lossless (huge files). 18-24 is high quality.


def crop_segment(
    vid_path: str,
    output_path: str,
    start_frame: int,
    end_frame: int,
    fps: float,
    W: int,
    H: int,
    X: int,
    Y: int,
):
    start_sec = start_frame / fps
    duration_sec = (end_frame - start_frame + 1) / fps

    crop_filter = f'crop={W}:{H}:{X}:{Y}'
    quiet_opts = ['-hide_banner', '-loglevel', 'error']

    command = [
        'ffmpeg',
        *quiet_opts,
        '-y',
        '-ss',
        f'{start_sec:.6f}',
        '-i',
        vid_path,
        '-t',
        f'{duration_sec:.6f}',
        '-vf',
        crop_filter,
    ]

    if ENCODER_TYPE == 'nvidia':
        encoder_opts = [
            '-c:v',
            'h264_nvenc',
            '-rc',
            'constqp',
            '-qp',
            str(QUALITY_SETTING),
            '-preset',
            'p1',
        ]
    elif ENCODER_TYPE == 'cpu':
        encoder_opts = [
            '-c:v',
            'libx264',
            '-crf',
            str(QUALITY_SETTING),
            '-preset',
            'superfast',
        ]
    else:
        raise ValueError(f'Unknown ENCODER_TYPE: {ENCODER_TYPE}')

    command.extend(encoder_opts)

    command.extend(
        [
            '-c:a',
            'copy',
            output_path,
        ]
    )

    try:
        subprocess.run(
            command,
            check=True,
            stdout=subprocess.DEVNULL,
            stderr=subprocess.PIPE,
            text=True,
        )
    except subprocess.CalledProcessError as e:
        print('--- FFMPEG ERROR ---')
        print(f'Error processing {vid_path} -> {output_path}')
        print(f'Command: {" ".join([shlex.quote(c) for c in command])}')
        print(f'Error Output: {e.stderr}')
        print('--------------------')


def get_w_h_fps(vid_path: str):
    command = [
        'ffprobe',
        '-v',
        'error',
        '-select_streams',
        'v:0',
        '-show_entries',
        'stream=width,height,r_frame_rate',
        '-of',
        'json',
        vid_path,
    ]

    try:
        result = subprocess.run(command, check=True, capture_output=True, text=True)
        metadata = json.loads(result.stdout)['streams'][0]

        w = int(metadata['width'])
        h = int(metadata['height'])

        # r_frame_rate is a fraction like "30/1" or "2997/100"
        num, den = metadata['r_frame_rate'].split('/')
        fps = float(num) / float(den)

        return w, h, fps

    except Exception as e:
        print(f'Error getting metadata for {vid_path}: {e}')
        print(f'ffprobe output: {result.stderr}')  # type: ignore
        raise


for root, _, files in os.walk('scraped'):
    for file in files:
        if not file.endswith('.json'):
            continue

        json_path = os.path.join(root, file)
        vid_path = json_path.replace('.json', '.mp4')
        base_out_path = json_path.replace('.json', '.seg.mp4')

        if not os.path.exists(vid_path):
            vid_path = json_path.replace('.json', '.mkv')
            base_out_path = json_path.replace('.json', '.seg.mkv')

        if os.path.exists(base_out_path):
            print(f'Skipping {vid_path}')
            continue

        try:
            w, h, fps = get_w_h_fps(vid_path)
        except Exception as e:
            print(f'Skipping {vid_path} due to metadata error: {e}')
            continue

        if h == 1080:
            X = 1522
            Y = 720
            W = 383
            H = 411
        elif h == 720:
            X = 1015
            Y = 480
            W = 255
            H = 274
        else:
            print(f'Skipping {vid_path}, height {h} not supported')
            continue

        with open(json_path) as f:
            segment = json.load(f)

        if not isinstance(segment, list):
            print(f'Processing {vid_path}...')
            crop_segment(
                vid_path,
                base_out_path,
                segment['start'],
                segment['end'],
                fps,
                W,
                H,
                X,
                Y,
            )
        else:
            print(f'Processing {vid_path} as {len(segment)} segments...')
            base_name, ext = os.path.splitext(base_out_path)

            for i, seg in enumerate(segment):
                out_path_i = f'{base_name}_{i}{ext}'

                if os.path.exists(out_path_i):
                    print(f'Skipping segment {i} (already exists)')
                    continue

                crop_segment(
                    vid_path, out_path_i, seg['start'], seg['end'], fps, W, H, X, Y
                )

        print(f'Finished {vid_path}')

### When manually cleaning up, see if chosen frame is ok...

In [ ]:
import matplotlib.pyplot as plt

idx = 1908
vid = 'i03z3ndRfAk.mkv'

cap, _, _ = get_vid_metadata(os.path.join('scraped', vid))
frame = get_frame(idx, cap)
frame = crop_interpreter(frame)

plt.imshow(frame)
plt.axis('off')
plt.show()